In [ ]:
# | default_exp routes.application

In [ ]:
# | exporti
from typing import Union
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [ ]:
# | export


@gd.route_function
async def get_applications(
    auth: dmda.DomoFullAuth,
    session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
    debug_api: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/executor/v1/applications/"

    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

#### sample implementation of get_applications

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


res = await get_applications(auth=token_auth)
pd.DataFrame(res.response)[0:5]

,applicationId,customerId,name,description,version,executionClass,verifyOwnership,strictValidation,imageLocation,authorities,internalAuthorities,created,updated,resources,k8s
0,25a97e0c-df6b-11eb-ba80-0242ac130004,mmmm-0012-0200,Toolkit: PDP Automation,PDP Automation,1.0.81_1,com.domo.executor.pdp.PDPAutomationApplication,False,True,972900829293.dkr.ecr.us-east-1.amazonaws.com/e...,"[pipeline.executor.job.create, dataset.manage]",[],1665079183000,1699383046000,"{'requests': {'memory': '3G'}, 'limits': {'mem...",True
1,4ddbf5d7-6441-4eb3-b5aa-97707cae2d2b,mmmm-0012-0200,Toolkit: Schema Management,Schema Management,1.0.25_1,com.domo.executor.schema.SchemaManagement,False,True,972900829293.dkr.ecr.us-east-1.amazonaws.com/e...,[dataset.admin],[],1665079188000,1693428986000,"{'requests': {'memory': '512Mi'}, 'limits': {'...",True
2,e37a6942-9c0f-485a-8288-4fe95e10b23d,mmmm-0012-0200,Toolkit: DataSet S3 Backup,DataSet S3 Backup,1.0.7_1,com.domo.executor.backup.DataSetBackupExecutor,False,True,972900829293.dkr.ecr.us-east-1.amazonaws.com/e...,"[dataset.admin, search.allentities.query]",[service.internal],1665079176000,1681325410000,"{'requests': {'memory': '256Mi'}, 'limits': {'...",True
3,b52f3c80-2642-4dcb-b874-b327326021b0,mmmm-0012-0200,Toolkit: User Automation,Toolkit: User Automation,1.51_2,com.domo.executor.userautomation.UserAutomatio...,False,True,972900829293.dkr.ecr.us-east-1.amazonaws.com/e...,"[pipeline.executor.job.create, dataset.manage,...",[],1665079192000,1693428995000,"{'requests': {'memory': '1G'}, 'limits': {'mem...",True
4,2f6573a5-97d8-4e27-b0fd-3c0f2313a3c8,mmmm-0012-0200,Toolkit: Group Management,Group Management,1.0.42_1,com.domo.executor.automation.GroupAutomationEx...,False,True,972900829293.dkr.ecr.us-east-1.amazonaws.com/e...,"[pipeline.executor.job.create, dataset.admin, ...",[],1665079181000,1682623726000,"{'requests': {'memory': '256Mi'}, 'limits': {'...",True


In [ ]:
# | export


@gd.route_function
async def get_application_by_id(
    auth: dmda.DomoFullAuth,
    application_id: str,
    session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None,
    debug_api: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop: int = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/executor/v1/applications/{application_id}"

    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
!nbqa black application.ipynb

All done! ✨ 🍰 ✨
1 file left unchanged.
